In [150]:
import json
import regex

import spacy
from spacy.matcher import Matcher
from spaczz.matcher import FuzzyMatcher
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd
from tqdm import tqdm

In [148]:
nlp = spacy.load('en_core_web_lg')
matcher = Matcher(nlp.vocab)

df = pd.read_csv("/Users/dclark171/projects/python/retailMarketAnalysis/data/reddit_submissions/all_submissions.csv")
sample = df.dropna(subset=['selftext'])[df['selftext'] != '[removed]'][df['selftext'] != '[deleted]'].sample(1000)

with open("/Users/dclark171/projects/python/retailMarketAnalysis/data/iex_data.json", 'r') as f:
    data = json.load(f)
    
with open('/Users/dclark171/projects/python/retailMarketAnalysis/data/english_dictionary.txt') as word_file:
    valid_words = set(word_file.read().split())

/Users/dclark171/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,10,11,20,23,35,63,71,72,77,84,85,86,87,89,90,91,92,93,94,95,97,100,101,103,104,105,106,108,110) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/dclark171/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [161]:
#tickers = list(pd.read_csv("/Users/dclark171/projects/python/retailMarketAnalysis/data/tickers.csv")['ticker'])
#tentative = []
for idx, x in tqdm(enumerate(tickers)):
    if x.lower in valid_words or wn.synsets(x):
        tentative.append(tickers.pop(idx))
    else:
        try:
            _ = wikipedia.page(x, auto_suggest=False)
        except wikipedia.exceptions.DisambiguationError:
            tentative.append(tickers.pop(idx))
        except wikipedia.exceptions.PageError:
            pass
print(len(tentative))

5it [00:00,  5.33it/s]/Users/dclark171/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/dclark171/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
5385it [25:06,  3.58it/s]

3205


In [168]:
ticker_company = {}

for x in tickers:
    if data.get(x):
        ticker_company[x] = (data[x]['companyName'], False)

for x in tentative:
    if data.get(x):
        ticker_company[x] = (data[x]['companyName'], True)
        
with open("data/ticker_cmp.json", 'w') as f:
    json.dump(ticker_company, f)


In [180]:
tickers = dict(filter(lambda s: False if s[1][1] else True, ticker_company.items()))
tentative = dict(filter(lambda s: True if s[1][1] else False, ticker_company.items()))

In [230]:
from spaczz.matcher import FuzzyMatcher

nlp2 = spacy.blank("en")

matcher2 = FuzzyMatcher(nlp2.vocab)

for k, v in tickers.items():
    cmp = v[0]
    if cmp:
        cmp = re.sub('[^A-Za-z0-9\s]+|incorporated|ETF|corporation|" \
                     "\sfund|\scorp|Ltd|\sinc|\sco|equity|\strust|\sSAA|\sLP|\splc|\s{2,}', '',
                     v[0], flags=re.IGNORECASE)
        if len(cmp) < 4:
            if len(cmp) <= 3:
                continue
            matcher2.add(k, [nlp2(cmp)], kwargs=[{"fuzzy_func": "simple", "min_r1":30, "min_r2": 100}])
            continue
        matcher2.add(k, [nlp2(cmp)], kwargs=[{"fuzzy_func": "simple", "min_r1":30, "min_r2": 90}])
for k, v in tentative.items():
    cmp = v[0]
    if cmp:
        cmp = re.sub('[^A-Za-z0-9\s]+|incorporated|ETF|corporation|" \
                     "\sfund|\scorp|Ltd|\sinc|\sco|equity|\strust|\sSAA|\sLP|\splc|\s{2,}', '',
                     v[0], flags=re.IGNORECASE)
        if len(cmp) < 4:
            if len(cmp) <= 3:
                continue
            matcher2.add(k, [nlp2(cmp)], kwargs=[{"fuzzy_func": "simple", "min_r1":30, "min_r2": 100}])
            continue
        matcher2.add(k, [nlp2(cmp)], kwargs=[{"fuzzy_func": "simple", "min_r1":30, "min_r2": 90}])



In [231]:
for i in sample['selftext'][:100]:
    doc = nlp2(i)
    matches = matcher2(doc)
    for match_id, start, end, ratio in matches:
        print(match_id, doc[start:end], ratio)

OFS of capital 95
INST structure 90
GES guess 100
UTL until 91
LASR night 91


KeyboardInterrupt: 

In [53]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

nlp = English()
ruler = EntityRuler(nlp)
patterns = [{"label": "TICKER", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

doc = nlp("Apple is opening its first big office in San Francisco.")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Apple', 'TICKER'), ('San Francisco', 'GPE')]


In [109]:
print(wikipedia.summary("USO"))

An unidentified flying object (UFO) is any aerial phenomenon that cannot immediately be identified or explained. Most UFOs are identified on investigation as conventional objects or phenomena. The term is widely used for claimed observations of extraterrestrial spacecraft.


In [ ]:
https://github.com/seatgeek/fuzzywuzzy
https://pypi.org/project/spaczz/#Fuzzy-Matcher
https://spacy.io/usage/rule-based-matching
https://spacy.io/usage/rule-based-matching#entityruler
https://spacy.io/usage/training    
https://medium.com/@b.terryjack/nlp-pretrained-named-entity-recognition-7caa5cd28d7b#:~:text=There%20are%20a%20good%20range,)%20API%20(e.g.%20GATE).